# App Rating Predictor
## Building an app rating predictor using data scraped from the android market

Import packages

In [2]:
import pandas as pd
import numpy as np
import datetime as dt
pd.set_option('max_columns', None) #Show all columns, or else some will just be "..."

In [3]:
df = pd.read_csv("./input/google_app_scrap.csv")
df.head()

,Unnamed: 0,APP_NAME,RATING,CATEGORY,COLLECTIONS,RATING_COUNT,1_STAR_RATINGS,2_STAR_RATINGS,3_STAR_RATINGS,4_STAR_RATINGS,5_STAR_RATINGS,REVIEW_COUNT,INSTALLS,MIN_INSTALLS,MAX_INSTALLS,FREE,PRICE,CURRENCY,SIZE,DEVELOPER,DEVELOPER_ADDRESS,CONTENT_RATING,FAMILY_GENRE,AD_SUPPORTED,IN_APP_PURCHASES,RELEASED_DATE,LAST_UPDATED,DAY_SCRAPED
0,0,Skincare Routine,4.510000,Beauty,topselling_paid,608.0,34,11,23,65,464,89.0,"10,000+",10000,17566,False,3.99,USD,16M,Mento Apps Ltd,23 Stoneleigh Avenue\nBN1 8NP,Everyone,NaN,False,False,"Aug 13, 2018",14/2/22,1/3/22
1,1,TurboScan: scan documents and receipts in PDF,4.800429,Business,topselling_paid,17641.0,340,75,189,1551,15479,2229.0,"100,000+",100000,259589,False,4.99,USD,Varies with device,Piksoft Inc.,"2500 Santa Clara Ave, Alameda, CA 94501",Everyone,NaN,False,False,"Apr 7, 2014",13/5/21,1/3/22
2,2,Package Disabler Pro (Owner),2.420000,Business,topselling_paid,742.0,421,36,50,7,221,163.0,"10,000+",10000,23412,False,1.49,USD,7.1M,policedeveloper,"Reading, Berkshire, RG6 3BT, United Kingdom",Everyone,NaN,False,False,"Nov 9, 2018",21/2/22,1/3/22
3,3,FX File Explorer (Plus License Key),4.284404,Business,topselling_paid,3452.0,315,94,315,283,2436,330.0,"100,000+",100000,105424,False,2.99,USD,Varies with device,"NextApp, Inc.","2549-B Eastbluff Drive #201\nNewport Beach, CA...",Everyone,NaN,False,False,"Sep 20, 2012",22/5/18,1/3/22
4,4,Jump Desktop (RDP & VNC),4.058394,Business,topselling_paid,3332.0,484,168,168,338,2163,994.0,"50,000+",50000,55881,False,4.99,USD,Varies with device,PhaseFive Systems,"POBOX 504905 DUBAI, UAE",Everyone,NaN,False,False,"Aug 17, 2011",4/8/18,1/3/22


## Data Cleaning

Basic statistical analysis with scraped data

In [4]:
df.describe(include="all")

,Unnamed: 0,APP_NAME,RATING,CATEGORY,COLLECTIONS,RATING_COUNT,1_STAR_RATINGS,2_STAR_RATINGS,3_STAR_RATINGS,4_STAR_RATINGS,5_STAR_RATINGS,REVIEW_COUNT,INSTALLS,MIN_INSTALLS,MAX_INSTALLS,FREE,PRICE,CURRENCY,SIZE,DEVELOPER,DEVELOPER_ADDRESS,CONTENT_RATING,FAMILY_GENRE,AD_SUPPORTED,IN_APP_PURCHASES,RELEASED_DATE,LAST_UPDATED,DAY_SCRAPED
count,23873.000000,23873,23857.000000,23873,23873,2.385700e+04,2.387300e+04,2.387300e+04,2.387300e+04,2.387300e+04,2.387300e+04,2.385700e+04,23873,2.387300e+04,2.387300e+04,23873,23873.000000,23873,23873,23873,19270,23873,0.0,23873,23873,23681,23873,23873
unique,NaN,18448,NaN,48,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22,NaN,NaN,2,NaN,1,670,12331,8590,5,NaN,2,2,3934,1598,1
top,NaN,Solitaire,NaN,Sports,topselling_free,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"1,000,000+",NaN,NaN,True,NaN,USD,Varies with device,My Town Games Ltd,"Ha-Rekhev St 1, Tel Aviv-Yafo",Everyone,NaN,False,True,"Nov 9, 2021",24/2/22,1/3/22
freq,NaN,28,NaN,946,10399,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4401,NaN,NaN,18318,NaN,23873,3190,140,140,17487,NaN,12217,14297,33,898,23873
mean,11936.000000,NaN,3.973446,NaN,NaN,3.805085e+05,4.255446e+04,9.158966e+03,1.775779e+04,4.151404e+04,2.692438e+05,1.745906e+04,NaN,1.896055e+07,3.076583e+07,NaN,1.443526,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,6891.685824,NaN,1.068241,NaN,NaN,3.609019e+06,7.755251e+05,9.935883e+04,1.713797e+05,3.554295e+05,2.385257e+06,1.173932e+05,NaN,2.228155e+08,2.837133e+08,NaN,9.265310,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,0.000000,NaN,0.000000,NaN,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NaN,0.000000e+00,0.000000e+00,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,5968.000000,NaN,3.880000,NaN,NaN,6.100000e+02,5.600000e+01,1.100000e+01,2.200000e+01,4.900000e+01,3.740000e+02,6.100000e+01,NaN,5.000000e+04,5.396800e+04,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,11936.000000,NaN,4.297787,NaN,NaN,8.103000e+03,6.560000e+02,1.790000e+02,3.250000e+02,7.320000e+02,5.409000e+03,5.410000e+02,NaN,5.000000e+05,7.966040e+05,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,17904.000000,NaN,4.560000,NaN,NaN,6.978300e+04,5.473000e+03,1.617000e+03,3.045000e+03,7.267000e+03,4.930900e+04,4.082000e+03,NaN,5.000000e+06,7.324750e+06,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23873 entries, 0 to 23872
Data columns (total 28 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         23873 non-null  int64  
 1   APP_NAME           23873 non-null  object 
 2   RATING             23857 non-null  float64
 3   CATEGORY           23873 non-null  object 
 4   COLLECTIONS        23873 non-null  object 
 5   RATING_COUNT       23857 non-null  float64
 6   1_STAR_RATINGS     23873 non-null  int64  
 7   2_STAR_RATINGS     23873 non-null  int64  
 8   3_STAR_RATINGS     23873 non-null  int64  
 9   4_STAR_RATINGS     23873 non-null  int64  
 10  5_STAR_RATINGS     23873 non-null  int64  
 11  REVIEW_COUNT       23857 non-null  float64
 12  INSTALLS           23873 non-null  object 
 13  MIN_INSTALLS       23873 non-null  int64  
 14  MAX_INSTALLS       23873 non-null  int64  
 15  FREE               23873 non-null  bool   
 16  PRICE              238

Remove columns "Unnamed: 0". "FAMILY_GENRE", "CURRENCY" as they contain same/empty values which will be useless in eda

In [6]:
df = df.drop(columns=['Unnamed: 0', 'FAMILY_GENRE', 'CURRENCY'])
df.columns

Index(['APP_NAME', 'RATING', 'CATEGORY', 'COLLECTIONS', 'RATING_COUNT',
       '1_STAR_RATINGS', '2_STAR_RATINGS', '3_STAR_RATINGS', '4_STAR_RATINGS',
       '5_STAR_RATINGS', 'REVIEW_COUNT', 'INSTALLS', 'MIN_INSTALLS',
       'MAX_INSTALLS', 'FREE', 'PRICE', 'SIZE', 'DEVELOPER',
       'DEVELOPER_ADDRESS', 'CONTENT_RATING', 'AD_SUPPORTED',
       'IN_APP_PURCHASES', 'RELEASED_DATE', 'LAST_UPDATED', 'DAY_SCRAPED'],
      dtype='object')

### Cleaning RATING_COUNT
The RATING_COUNT column has inconsistent values, change the values in RATING_COUNT column to the sum of 1_STAR_RATING, 2_STAR_RATING, 3_STAR_RATING, 4_STAR_RATING, 5_STAR_RATING columns

In [7]:
df['RATING_COUNT'] = df[['1_STAR_RATINGS', '2_STAR_RATINGS', '3_STAR_RATINGS', '4_STAR_RATINGS', '5_STAR_RATINGS']].sum(axis=1)
df.head()

,APP_NAME,RATING,CATEGORY,COLLECTIONS,RATING_COUNT,1_STAR_RATINGS,2_STAR_RATINGS,3_STAR_RATINGS,4_STAR_RATINGS,5_STAR_RATINGS,REVIEW_COUNT,INSTALLS,MIN_INSTALLS,MAX_INSTALLS,FREE,PRICE,SIZE,DEVELOPER,DEVELOPER_ADDRESS,CONTENT_RATING,AD_SUPPORTED,IN_APP_PURCHASES,RELEASED_DATE,LAST_UPDATED,DAY_SCRAPED
0,Skincare Routine,4.510000,Beauty,topselling_paid,597,34,11,23,65,464,89.0,"10,000+",10000,17566,False,3.99,16M,Mento Apps Ltd,23 Stoneleigh Avenue\nBN1 8NP,Everyone,False,False,"Aug 13, 2018",14/2/22,1/3/22
1,TurboScan: scan documents and receipts in PDF,4.800429,Business,topselling_paid,17634,340,75,189,1551,15479,2229.0,"100,000+",100000,259589,False,4.99,Varies with device,Piksoft Inc.,"2500 Santa Clara Ave, Alameda, CA 94501",Everyone,False,False,"Apr 7, 2014",13/5/21,1/3/22
2,Package Disabler Pro (Owner),2.420000,Business,topselling_paid,735,421,36,50,7,221,163.0,"10,000+",10000,23412,False,1.49,7.1M,policedeveloper,"Reading, Berkshire, RG6 3BT, United Kingdom",Everyone,False,False,"Nov 9, 2018",21/2/22,1/3/22
3,FX File Explorer (Plus License Key),4.284404,Business,topselling_paid,3443,315,94,315,283,2436,330.0,"100,000+",100000,105424,False,2.99,Varies with device,"NextApp, Inc.","2549-B Eastbluff Drive #201\nNewport Beach, CA...",Everyone,False,False,"Sep 20, 2012",22/5/18,1/3/22
4,Jump Desktop (RDP & VNC),4.058394,Business,topselling_paid,3321,484,168,168,338,2163,994.0,"50,000+",50000,55881,False,4.99,Varies with device,PhaseFive Systems,"POBOX 504905 DUBAI, UAE",Everyone,False,False,"Aug 17, 2011",4/8/18,1/3/22


### Cleaning INSTALL
The INSTALL column is filled with string values, remove '+' sign from the string and convert the string in numerical

In [8]:
df["INSTALLS"] = df["INSTALLS"].str.replace("+","")
df["INSTALLS"] = df["INSTALLS"].str.replace(",","")
df['INSTALLS'] = df['INSTALLS'].astype(str).astype('int64')
df["INSTALLS"]

<ipython-input-8-d1f641f37301>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df["INSTALLS"] = df["INSTALLS"].str.replace("+","")


0          10000
1         100000
2          10000
3         100000
4          50000
          ...   
23868    1000000
23869     100000
23870      50000
23871      10000
23872     100000
Name: INSTALLS, Length: 23873, dtype: int64

### Cleaning FREE, AD_SUPPORTED & IN_APP_PURCHASES
FREE , AD_SUPPORTED  & IN_APP_PURCHASES contains boolean values, convert them to numerical by mapping True to 1 and False to 0

In [9]:
df['FREE'] = df["FREE"].astype(int)
df['AD_SUPPORTED'] = df["AD_SUPPORTED"].astype(int)
df['IN_APP_PURCHASES'] = df["IN_APP_PURCHASES"].astype(int)
print("Data type of FREE:", df.dtypes["FREE"])
print("Data type of AD_SUPPORTED:", df.dtypes["AD_SUPPORTED"])
print("Data type of IN_APP_PURCHASES:", df.dtypes["IN_APP_PURCHASES"])
df.head()

Data type of FREE: int32
Data type of AD_SUPPORTED: int32
Data type of IN_APP_PURCHASES: int32


,APP_NAME,RATING,CATEGORY,COLLECTIONS,RATING_COUNT,1_STAR_RATINGS,2_STAR_RATINGS,3_STAR_RATINGS,4_STAR_RATINGS,5_STAR_RATINGS,REVIEW_COUNT,INSTALLS,MIN_INSTALLS,MAX_INSTALLS,FREE,PRICE,SIZE,DEVELOPER,DEVELOPER_ADDRESS,CONTENT_RATING,AD_SUPPORTED,IN_APP_PURCHASES,RELEASED_DATE,LAST_UPDATED,DAY_SCRAPED
0,Skincare Routine,4.510000,Beauty,topselling_paid,597,34,11,23,65,464,89.0,10000,10000,17566,0,3.99,16M,Mento Apps Ltd,23 Stoneleigh Avenue\nBN1 8NP,Everyone,0,0,"Aug 13, 2018",14/2/22,1/3/22
1,TurboScan: scan documents and receipts in PDF,4.800429,Business,topselling_paid,17634,340,75,189,1551,15479,2229.0,100000,100000,259589,0,4.99,Varies with device,Piksoft Inc.,"2500 Santa Clara Ave, Alameda, CA 94501",Everyone,0,0,"Apr 7, 2014",13/5/21,1/3/22
2,Package Disabler Pro (Owner),2.420000,Business,topselling_paid,735,421,36,50,7,221,163.0,10000,10000,23412,0,1.49,7.1M,policedeveloper,"Reading, Berkshire, RG6 3BT, United Kingdom",Everyone,0,0,"Nov 9, 2018",21/2/22,1/3/22
3,FX File Explorer (Plus License Key),4.284404,Business,topselling_paid,3443,315,94,315,283,2436,330.0,100000,100000,105424,0,2.99,Varies with device,"NextApp, Inc.","2549-B Eastbluff Drive #201\nNewport Beach, CA...",Everyone,0,0,"Sep 20, 2012",22/5/18,1/3/22
4,Jump Desktop (RDP & VNC),4.058394,Business,topselling_paid,3321,484,168,168,338,2163,994.0,50000,50000,55881,0,4.99,Varies with device,PhaseFive Systems,"POBOX 504905 DUBAI, UAE",Everyone,0,0,"Aug 17, 2011",4/8/18,1/3/22


### Cleaning SIZE
SIZE contains string objects, convert them to number for eda later. <br/>
For example: 12M -> 12000000, 12K -> 12000, varies with device -> -1 (drop this later)

In [10]:
df["SIZE"] = df["SIZE"].str.replace("Varies with device","-1") #replacing "Varies with device" with -1 
df["SIZE"] = df["SIZE"].str.replace(",","") #replacing ',' with nothing

In [11]:
#changing k to (10**3), M to (10**6) and G to (10**9)
df["SIZE"] = (df["SIZE"].replace(r'[kMG]+$', '', regex=True).astype(float) * \
                   df["SIZE"].str.extract(r'[\d\.]+([kMG]+)', expand=False)
                   .fillna(1)
                   .replace(['k','M','G'], [10**3, 10**6,10**9]).astype(int))
df['SIZE'] = df['SIZE'].astype('int64') #converting float to int64
df['SIZE']

0        16000000
1              -1
2         7100000
3              -1
4              -1
           ...   
23868    24000000
23869     6700000
23870          -1
23871    26000000
23872    10000000
Name: SIZE, Length: 23873, dtype: int64

Convert all -1 values to NaN, we will fill this later

In [12]:
df.loc[df['SIZE'] == -1] = np.nan
df['SIZE']

0        16000000.0
1               NaN
2         7100000.0
3               NaN
4               NaN
            ...    
23868    24000000.0
23869     6700000.0
23870           NaN
23871    26000000.0
23872    10000000.0
Name: SIZE, Length: 23873, dtype: float64

### Cleaning DAY_RELEASED
The format of dates in DAY_RELEASED, LAST_UPDATED and DAY_SCRAPED are inconsistent, convert both of them to the format "dd/mm/yyyy"

In [13]:
df['RELEASED_DATE'] = pd.to_datetime(df['RELEASED_DATE'])
df['LAST_UPDATED'] = pd.to_datetime(df['LAST_UPDATED'], format="%d/%m/%y")
df['DAY_SCRAPED'] = pd.to_datetime(df['DAY_SCRAPED'], format="%d/%m/%y")
df['RELEASED_DATE'] = df['RELEASED_DATE'].dt.strftime('%d/%m/%Y')
df['LAST_UPDATED'] = df['LAST_UPDATED'].dt.strftime('%d/%m/%Y')
df['DAY_SCRAPED'] = df['DAY_SCRAPED'].dt.strftime('%d/%m/%Y')
df.head(5)

,APP_NAME,RATING,CATEGORY,COLLECTIONS,RATING_COUNT,1_STAR_RATINGS,2_STAR_RATINGS,3_STAR_RATINGS,4_STAR_RATINGS,5_STAR_RATINGS,REVIEW_COUNT,INSTALLS,MIN_INSTALLS,MAX_INSTALLS,FREE,PRICE,SIZE,DEVELOPER,DEVELOPER_ADDRESS,CONTENT_RATING,AD_SUPPORTED,IN_APP_PURCHASES,RELEASED_DATE,LAST_UPDATED,DAY_SCRAPED
0,Skincare Routine,4.51,Beauty,topselling_paid,597.0,34.0,11.0,23.0,65.0,464.0,89.0,10000.0,10000.0,17566.0,0.0,3.99,16000000.0,Mento Apps Ltd,23 Stoneleigh Avenue\nBN1 8NP,Everyone,0.0,0.0,13/08/2018,14/02/2022,01/03/2022
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Package Disabler Pro (Owner),2.42,Business,topselling_paid,735.0,421.0,36.0,50.0,7.0,221.0,163.0,10000.0,10000.0,23412.0,0.0,1.49,7100000.0,policedeveloper,"Reading, Berkshire, RG6 3BT, United Kingdom",Everyone,0.0,0.0,09/11/2018,21/02/2022,01/03/2022
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Dealing with NULL Values

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23873 entries, 0 to 23872
Data columns (total 25 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   APP_NAME           20683 non-null  object 
 1   RATING             20667 non-null  float64
 2   CATEGORY           20683 non-null  object 
 3   COLLECTIONS        20683 non-null  object 
 4   RATING_COUNT       20683 non-null  float64
 5   1_STAR_RATINGS     20683 non-null  float64
 6   2_STAR_RATINGS     20683 non-null  float64
 7   3_STAR_RATINGS     20683 non-null  float64
 8   4_STAR_RATINGS     20683 non-null  float64
 9   5_STAR_RATINGS     20683 non-null  float64
 10  REVIEW_COUNT       20667 non-null  float64
 11  INSTALLS           20683 non-null  float64
 12  MIN_INSTALLS       20683 non-null  float64
 13  MAX_INSTALLS       20683 non-null  float64
 14  FREE               20683 non-null  float64
 15  PRICE              20683 non-null  float64
 16  SIZE               206

There are null values in columns "REVIEW_COUNT", "RATING", "RATING_COUNT", "RELEASED_DATE", for "REVIEW_COUNT", "RATING", "RATING_COUNT", the values are null value most likely because the app is too new and no customers have done any review on them, so remove them from the dataset. 

In [15]:
df = df.dropna(axis=0, subset=['REVIEW_COUNT', 'RATING', 'RATING_COUNT'])
df = df.reset_index()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20667 entries, 0 to 20666
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   index              20667 non-null  int64  
 1   APP_NAME           20667 non-null  object 
 2   RATING             20667 non-null  float64
 3   CATEGORY           20667 non-null  object 
 4   COLLECTIONS        20667 non-null  object 
 5   RATING_COUNT       20667 non-null  float64
 6   1_STAR_RATINGS     20667 non-null  float64
 7   2_STAR_RATINGS     20667 non-null  float64
 8   3_STAR_RATINGS     20667 non-null  float64
 9   4_STAR_RATINGS     20667 non-null  float64
 10  5_STAR_RATINGS     20667 non-null  float64
 11  REVIEW_COUNT       20667 non-null  float64
 12  INSTALLS           20667 non-null  float64
 13  MIN_INSTALLS       20667 non-null  float64
 14  MAX_INSTALLS       20667 non-null  float64
 15  FREE               20667 non-null  float64
 16  PRICE              206

As for "RELEASED_DATE", fill the null values with the median. 

But first we create to new columns to replace "RELEASED_DATE" and "LAST_UPDATED" to contain numerical values instead of Date object.

In [16]:
for i in range(df.shape[0]):
    if (pd.isnull(df.loc[i, "LAST_UPDATED"])):
        df.loc[i, "DAYS_SINCE_UPDATE"] = np.nan
        continue
    date_from = pd.to_datetime(df.loc[i, "LAST_UPDATED"], format="%d/%m/%Y").date()
    date_till = pd.to_datetime(df.loc[i, "DAY_SCRAPED"], format="%d/%m/%Y").date()
    date_elasped = np.int64((date_till-date_from).days)
    df.loc[i, "DAYS_SINCE_UPDATE"] = date_elasped
    
    if (pd.isnull(df.loc[i, "RELEASED_DATE"])):
        df.loc[i, "DAYS_SINCE_RELEASED"] = np.nan
        continue
    
    date_from = pd.to_datetime(df.loc[i, "RELEASED_DATE"], format="%d/%m/%Y").date()
    date_till = pd.to_datetime(df.loc[i, "DAY_SCRAPED"], format="%d/%m/%Y").date()
    date_elasped = np.int64((date_till-date_from).days)
    df.loc[i, "DAYS_SINCE_RELEASED"] = date_elasped
    
df.head()

,index,APP_NAME,RATING,CATEGORY,COLLECTIONS,RATING_COUNT,1_STAR_RATINGS,2_STAR_RATINGS,3_STAR_RATINGS,4_STAR_RATINGS,5_STAR_RATINGS,REVIEW_COUNT,INSTALLS,MIN_INSTALLS,MAX_INSTALLS,FREE,PRICE,SIZE,DEVELOPER,DEVELOPER_ADDRESS,CONTENT_RATING,AD_SUPPORTED,IN_APP_PURCHASES,RELEASED_DATE,LAST_UPDATED,DAY_SCRAPED,DAYS_SINCE_UPDATE,DAYS_SINCE_RELEASED
0,0,Skincare Routine,4.51,Beauty,topselling_paid,597.0,34.0,11.0,23.0,65.0,464.0,89.0,10000.0,10000.0,17566.0,0.0,3.99,16000000.0,Mento Apps Ltd,23 Stoneleigh Avenue\nBN1 8NP,Everyone,0.0,0.0,13/08/2018,14/02/2022,01/03/2022,15.0,1296.0
1,2,Package Disabler Pro (Owner),2.42,Business,topselling_paid,735.0,421.0,36.0,50.0,7.0,221.0,163.0,10000.0,10000.0,23412.0,0.0,1.49,7100000.0,policedeveloper,"Reading, Berkshire, RG6 3BT, United Kingdom",Everyone,0.0,0.0,09/11/2018,21/02/2022,01/03/2022,8.0,1208.0
2,5,"Tiny Scanner PRO, OCR",0.00,Business,topselling_paid,0.0,0.0,0.0,0.0,0.0,0.0,0.0,500.0,500.0,589.0,0.0,4.49,81000000.0,Mayno Apps,Güvenevler Mah.Keklik Cad.\nTürkiye,Everyone,0.0,0.0,25/06/2020,22/02/2022,01/03/2022,7.0,614.0
3,6,Payroll Guru,2.95,Business,topselling_paid,184.0,59.0,22.0,24.0,27.0,52.0,120.0,5000.0,5000.0,7112.0,0.0,4.99,108000.0,"Payrollguru, Inc.","12463 Rancho Bernardo Rd. #206, San Diego, CA ...",Everyone,0.0,0.0,17/04/2011,01/01/2022,01/03/2022,59.0,3971.0
4,7,Zanco Pro,0.00,Business,topselling_paid,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,100.0,113.0,0.0,1.99,9900000.0,Beyouacademy,NaN,Everyone,0.0,0.0,06/01/2022,19/02/2022,01/03/2022,10.0,54.0


Fill na values with the median

In [17]:
median_dsr = df["DAYS_SINCE_RELEASED"].median()
median_dsu = df["DAYS_SINCE_UPDATE"].median()
df["DAYS_SINCE_RELEASED"].fillna(median_dsr, inplace=True)
df["DAYS_SINCE_UPDATE"].fillna(median_dsu, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20667 entries, 0 to 20666
Data columns (total 28 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   index                20667 non-null  int64  
 1   APP_NAME             20667 non-null  object 
 2   RATING               20667 non-null  float64
 3   CATEGORY             20667 non-null  object 
 4   COLLECTIONS          20667 non-null  object 
 5   RATING_COUNT         20667 non-null  float64
 6   1_STAR_RATINGS       20667 non-null  float64
 7   2_STAR_RATINGS       20667 non-null  float64
 8   3_STAR_RATINGS       20667 non-null  float64
 9   4_STAR_RATINGS       20667 non-null  float64
 10  5_STAR_RATINGS       20667 non-null  float64
 11  REVIEW_COUNT         20667 non-null  float64
 12  INSTALLS             20667 non-null  float64
 13  MIN_INSTALLS         20667 non-null  float64
 14  MAX_INSTALLS         20667 non-null  float64
 15  FREE                 20667 non-null 

In [18]:
df['DAYS_SINCE_RELEASED'] = df['DAYS_SINCE_RELEASED'].astype('int64')
df['DAYS_SINCE_UPDATE'] = df['DAYS_SINCE_UPDATE'].astype('int64')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20667 entries, 0 to 20666
Data columns (total 28 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   index                20667 non-null  int64  
 1   APP_NAME             20667 non-null  object 
 2   RATING               20667 non-null  float64
 3   CATEGORY             20667 non-null  object 
 4   COLLECTIONS          20667 non-null  object 
 5   RATING_COUNT         20667 non-null  float64
 6   1_STAR_RATINGS       20667 non-null  float64
 7   2_STAR_RATINGS       20667 non-null  float64
 8   3_STAR_RATINGS       20667 non-null  float64
 9   4_STAR_RATINGS       20667 non-null  float64
 10  5_STAR_RATINGS       20667 non-null  float64
 11  REVIEW_COUNT         20667 non-null  float64
 12  INSTALLS             20667 non-null  float64
 13  MIN_INSTALLS         20667 non-null  float64
 14  MAX_INSTALLS         20667 non-null  float64
 15  FREE                 20667 non-null 

Drop RELEASED_DATE, LAST_UPDATED and DAY_SCRAPED   

In [19]:
df = df.drop(columns=['RELEASED_DATE', 'LAST_UPDATED', 'DAY_SCRAPED'])
df.columns

Index(['index', 'APP_NAME', 'RATING', 'CATEGORY', 'COLLECTIONS',
       'RATING_COUNT', '1_STAR_RATINGS', '2_STAR_RATINGS', '3_STAR_RATINGS',
       '4_STAR_RATINGS', '5_STAR_RATINGS', 'REVIEW_COUNT', 'INSTALLS',
       'MIN_INSTALLS', 'MAX_INSTALLS', 'FREE', 'PRICE', 'SIZE', 'DEVELOPER',
       'DEVELOPER_ADDRESS', 'CONTENT_RATING', 'AD_SUPPORTED',
       'IN_APP_PURCHASES', 'DAYS_SINCE_UPDATE', 'DAYS_SINCE_RELEASED'],
      dtype='object')

Filling null values of SIZE with median

In [20]:
median_size = df["SIZE"].median()
df["SIZE"].fillna(median_size, inplace=True)
df["SIZE"]

0        16000000.0
1         7100000.0
2        81000000.0
3          108000.0
4         9900000.0
            ...    
20662    76000000.0
20663    24000000.0
20664     6700000.0
20665    26000000.0
20666    10000000.0
Name: SIZE, Length: 20667, dtype: float64

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20667 entries, 0 to 20666
Data columns (total 25 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   index                20667 non-null  int64  
 1   APP_NAME             20667 non-null  object 
 2   RATING               20667 non-null  float64
 3   CATEGORY             20667 non-null  object 
 4   COLLECTIONS          20667 non-null  object 
 5   RATING_COUNT         20667 non-null  float64
 6   1_STAR_RATINGS       20667 non-null  float64
 7   2_STAR_RATINGS       20667 non-null  float64
 8   3_STAR_RATINGS       20667 non-null  float64
 9   4_STAR_RATINGS       20667 non-null  float64
 10  5_STAR_RATINGS       20667 non-null  float64
 11  REVIEW_COUNT         20667 non-null  float64
 12  INSTALLS             20667 non-null  float64
 13  MIN_INSTALLS         20667 non-null  float64
 14  MAX_INSTALLS         20667 non-null  float64
 15  FREE                 20667 non-null 

Cleaning Developer Address

Remove # from each row in developer address column

In [22]:
df = df.drop(["index"], axis = 1)
for i in range(df.shape[0]):
    developer_addr = df.loc[i, "DEVELOPER_ADDRESS"]
    if (pd.isnull(developer_addr)):
        continue
    df.loc[i, "DEVELOPER_ADDRESS"] = developer_addr.replace("#", " ")
df.head()

,APP_NAME,RATING,CATEGORY,COLLECTIONS,RATING_COUNT,1_STAR_RATINGS,2_STAR_RATINGS,3_STAR_RATINGS,4_STAR_RATINGS,5_STAR_RATINGS,REVIEW_COUNT,INSTALLS,MIN_INSTALLS,MAX_INSTALLS,FREE,PRICE,SIZE,DEVELOPER,DEVELOPER_ADDRESS,CONTENT_RATING,AD_SUPPORTED,IN_APP_PURCHASES,DAYS_SINCE_UPDATE,DAYS_SINCE_RELEASED
0,Skincare Routine,4.51,Beauty,topselling_paid,597.0,34.0,11.0,23.0,65.0,464.0,89.0,10000.0,10000.0,17566.0,0.0,3.99,16000000.0,Mento Apps Ltd,23 Stoneleigh Avenue\nBN1 8NP,Everyone,0.0,0.0,15,1296
1,Package Disabler Pro (Owner),2.42,Business,topselling_paid,735.0,421.0,36.0,50.0,7.0,221.0,163.0,10000.0,10000.0,23412.0,0.0,1.49,7100000.0,policedeveloper,"Reading, Berkshire, RG6 3BT, United Kingdom",Everyone,0.0,0.0,8,1208
2,"Tiny Scanner PRO, OCR",0.00,Business,topselling_paid,0.0,0.0,0.0,0.0,0.0,0.0,0.0,500.0,500.0,589.0,0.0,4.49,81000000.0,Mayno Apps,Güvenevler Mah.Keklik Cad.\nTürkiye,Everyone,0.0,0.0,7,614
3,Payroll Guru,2.95,Business,topselling_paid,184.0,59.0,22.0,24.0,27.0,52.0,120.0,5000.0,5000.0,7112.0,0.0,4.99,108000.0,"Payrollguru, Inc.","12463 Rancho Bernardo Rd. 206, San Diego, CA ...",Everyone,0.0,0.0,59,3971
4,Zanco Pro,0.00,Business,topselling_paid,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,100.0,113.0,0.0,1.99,9900000.0,Beyouacademy,NaN,Everyone,0.0,0.0,10,54


In [25]:
df.sample(20)

,APP_NAME,RATING,CATEGORY,COLLECTIONS,RATING_COUNT,1_STAR_RATINGS,2_STAR_RATINGS,3_STAR_RATINGS,4_STAR_RATINGS,5_STAR_RATINGS,REVIEW_COUNT,INSTALLS,MIN_INSTALLS,MAX_INSTALLS,FREE,PRICE,SIZE,DEVELOPER,DEVELOPER_ADDRESS,CONTENT_RATING,AD_SUPPORTED,IN_APP_PURCHASES,DAYS_SINCE_UPDATE,DAYS_SINCE_RELEASED
10848,Training Premiere Pro CS6 & CC,0.000000,Video Players & Editors,topgrossing,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,100.0,122.0,0.0,74.99,589000.0,Netframes,"1115 Broadway\n12th Floor\nNew York, NY 10010",Everyone,0.0,0.0,2922,2923
17467,"Cook, Serve, Delicious!",4.660000,Simulation,topselling_paid,430.0,11.0,7.0,11.0,43.0,358.0,87.0,10000.0,10000.0,11272.0,0.0,3.99,53000000.0,Vertigo Gaming,"PO Box 392\nOdessa, Texas 79760\nUnited States",Teen,0.0,0.0,1731,1754
19947,Wordle - Word Puzzle Game,2.600000,Word,topselling_free,17.0,7.0,0.0,3.0,7.0,0.0,5.0,10000.0,10000.0,10996.0,1.0,0.00,3800000.0,Mukesh Yadav⚡,NaN,Everyone,0.0,0.0,19,20
10857,Bravo Cleaner: Speed Booster,4.305232,Tools,topselling_free,4377.0,118.0,91.0,912.0,472.0,2784.0,382.0,1000000.0,1000000.0,2639915.0,1.0,0.00,12000000.0,Bravo GmbH,NaN,Everyone,1.0,1.0,5,161
18295,Word Cookies! ®,4.561942,Word,topgrossing,2268246.0,76669.0,37409.0,106453.0,361806.0,1685909.0,543068.0,10000000.0,10000000.0,45623082.0,1.0,0.00,125000000.0,BitMango,"경기도 성남시 분당구 대왕판교로645번길 14, 3층 (우편번호 13487)",Everyone,1.0,1.0,7,1909
8477,Trinity Health MyChart,4.447619,Medical,topselling_free,554.0,31.0,10.0,31.0,84.0,398.0,272.0,100000.0,100000.0,123894.0,1.0,0.00,41000000.0,Trinity Information Services Web Development,NaN,Everyone,0.0,0.0,24,782
2669,AlphaNovel - Romance Stories,4.738318,Books & Reference,topgrossing,352.0,2.0,0.0,12.0,52.0,286.0,8.0,5000.0,5000.0,5621.0,1.0,0.00,10000000.0,Brailion Ventures Limited,"Spyrou Kyprianou, 79, Protopapas bldg, 2nd flo...",Teen,0.0,1.0,12,154
10575,Animal Hair Salon Australia,3.888889,Educational,topselling_free,40601.0,7319.0,1697.0,3328.0,4065.0,24192.0,1914.0,10000000.0,10000000.0,25275882.0,1.0,0.00,86000000.0,TutoTOONS,Summit House\n4-5 Mitchell Street\nEdinburgh\n...,Everyone,1.0,1.0,168,1519
18801,Friends & Dragons,4.370892,Role Playing,topgrossing,9178.0,365.0,149.0,623.0,2608.0,5433.0,582.0,100000.0,100000.0,179436.0,1.0,0.00,137000000.0,Playsome Ltd,Urho Kekkosen Katu 7B\n00100 Helsinki\nFinland,Everyone 10+,0.0,1.0,12,565
1255,EVgo,1.710000,Auto & Vehicles,topselling_free,685.0,474.0,102.0,27.0,0.0,82.0,471.0,100000.0,100000.0,152679.0,1.0,0.00,21000000.0,EVgo,NaN,Everyone,0.0,0.0,14,1644


In [46]:
#exporting the cleaned data into a csv file
file_path = "./output/google_app_scrap_cleaned.csv"
df.to_csv(path_or_buf=file_path, index=False);